Let's create a holdem game using test-driven-development

In [1]:
from NLHoldem import Game
from HandComperator import strength

In [9]:
game = Game()
game.new_hand()
hole_0 = game.positions[0].holecards
hole_1 = game.positions[1].holecards
community = game.deck[4:9]
community_lst = [(card.value+2,card.suit) for card in community]
hole_0_lst = [(hole_0[0].value+2, hole_0[0].suit),(hole_0[1].value+2, hole_0[1].suit)]
hole_1_lst = [(hole_1[0].value+2, hole_1[0].suit),(hole_1[1].value+2, hole_1[1].suit)]
hand_0_lst = hole_0_lst + community_lst
hand_1_lst = hole_1_lst + community_lst


In [10]:
print(hand_0_lst)
print(strength(hand_0_lst))

[(2, 3), (13, 1), (7, 1), (11, 3), (2, 0), (2, 1), (4, 0)]
3.0213110000000003


In [9]:
1+1e-2*substrength[0]+1e-4*substrength[1]+1e-6*substrength[2]+1e-8*substrength[3]

1.0514131000000002

In [31]:
i = 0
for combo in combos:
    i+=1
print(i)

133784560


In [8]:
players = [0,1,2,3]
next_to_act = collections.deque(players)
next_to_act.reverse()
next_to_act

deque([3, 2, 1, 0])

In [10]:
next_to_act.pop()

0

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('data.csv',sep=";")
data

,ID,Year_of_Birth,Highest_Qualification,Marital_Status,Income,Kids_home,Teen_home,Amount_Wines,Amount_Fruits,Amount_Beers,Amount_Meat,Amount_Vegetables,Amount_Fish,No._WebPurchases,No._CatalogPurchases,No._StorePurchases,AcceptedCmp2
0,5524,1957,Graduation,Single,58138.0,0,0,635,88,546,172,88,88,8,10,4,0
1,2174,1954,Graduation,Single,46344.0,1,1,11,1,6,2,1,6,1,1,2,0
2,4141,1965,Graduation,Together,71613.0,0,0,426,49,127,111,21,42,8,2,10,0
3,6182,1984,Graduation,Together,26646.0,1,0,11,4,20,10,3,5,2,0,4,0
4,5324,1981,PhD,Married,58293.0,1,0,173,43,118,46,27,15,5,3,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,709,43,182,42,118,247,9,3,4,0
2236,4001,1946,PhD,Together,64014.0,2,1,406,0,30,0,0,8,8,2,5,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,908,48,217,32,12,24,2,3,13,0
2238,8235,1956,Master,Together,69245.0,0,1,428,30,214,80,30,61,6,5,10,0


In [6]:
data['spent_total'] = data[['Amount_Wines', 'Amount_Fruits', 'Amount_Beers', 'Amount_Meat', 'Amount_Vegetables', 'Amount_Vegetables']].sum(axis=1)

In [7]:
import scipy.stats as stats

In [30]:
no_kids=data['spent_total'][(data.Kids_home==0) & (data.Teen_home==0) & (data.Marital_Status=='Married')]
kids= data['spent_total'][((data.Kids_home>0) | (data.Teen_home>0))& (data.Marital_Status=='Married')]

In [31]:
stats.ttest_ind(no_kids,kids, equal_var=False)

Ttest_indResult(statistic=13.287586534275183, pvalue=3.917722111656694e-32)

In [32]:
no_kids.mean()

1040.5478260869565

In [33]:
kids.mean()

405.67507886435334

In [15]:
data.Income[data.Kids_home==0].mean()

61779.85814497272

In [16]:
data.Income[data.Kids_home==1].mean()

39138.07666290868

In [26]:
stats.levene(kids,no_kids)

LeveneResult(statistic=146.53325365962857, pvalue=1.0205371700134244e-32)